### Turing Machines

In [27]:
%run core.ipynb

In [167]:
from array import array


class Tape:

    def __init__(self, values):
        self.values = array("I", values)
        self.pos = 0
        
    def __repr__(self):
        return " ".join([f"{v}" for v in self.values[:self.pos]] + 
                        [_blu(self.values[self.pos])] + 
                        [f"{v}" for v in self.values[self.pos + 1:]])
    
    def __getitem__(self, i):
        return self.values[i]
    
    def __iter__(self):
        return iter(self.values)
    
    def __len__(self):
        return len(self.values)
    
    def read(self):
        return self.values[self.pos]

    def write(self, v):
        self.values[self.pos] = v

    def move(self, mv):
        if mv == "L":
            if self.pos == 0:
                self.values.insert(0, 0)
            else:
                self.pos -= 1
        else:
            if self.pos == len(self.values) - 1:
                self.values.append(0)
            self.pos += 1


In [168]:
class TuringMachine:
    
    def __init__(self, instructions):
        self.instructions = instructions
        
    def __getitem__(self, i):
        return self.instructions[i]

    def __call__(self, tape, max_iterations=None):
        i, instr, mv = 0, 0, None
        while not mv == "S":
            if max_iterations is not None:
                if i > max_iterations:
                    raise StopIteration(f"max iterations {max_iterations} reached")
            value = tape.read()
            next_instr, new_value, mv = self.instructions[instr][value]
            yield instr, next_instr, value, new_value, mv
            tape.write(new_value)
            tape.move(mv)
            instr = next_instr
            i += 1


In [169]:
def run_turing_machine(tm, tape, max_iterations=None):
    for instr, _, value, new_value, move in tm(tape, max_iterations=max_iterations):
        print(f"{instr}: {value} -> {new_value} mv={move}:", tape)
    print()
    print(f"result: {tape}")

In [170]:
UN_PLUS_ONE = {
    0: {
        0: (0, 0, "R"), 
        1: (1, 1, "R"),
    },
    1: {
        0: (0, 1, "S"), 
        1: (1, 1, "R"),
    },
}
un_plus_one = TuringMachine(UN_PLUS_ONE)

In [171]:
tape = Tape([0, 0, 1, 1, 1, 0, 0])
run_turing_machine(un_plus_one, tape)

0: 0 -> 0 mv=R: 0 0 1 1 1 0 0
0: 0 -> 0 mv=R: 0 0 1 1 1 0 0
0: 1 -> 1 mv=R: 0 0 1 1 1 0 0
1: 1 -> 1 mv=R: 0 0 1 1 1 0 0
1: 1 -> 1 mv=R: 0 0 1 1 1 0 0
1: 0 -> 1 mv=S: 0 0 1 1 1 0 0

result: 0 0 1 1 1 1 0


In [172]:
tape = Tape([0, 1, 1, 1, 1, 1, 0, 0])
run_turing_machine(un_plus_one, tape)

0: 0 -> 0 mv=R: 0 1 1 1 1 1 0 0
0: 1 -> 1 mv=R: 0 1 1 1 1 1 0 0
1: 1 -> 1 mv=R: 0 1 1 1 1 1 0 0
1: 1 -> 1 mv=R: 0 1 1 1 1 1 0 0
1: 1 -> 1 mv=R: 0 1 1 1 1 1 0 0
1: 1 -> 1 mv=R: 0 1 1 1 1 1 0 0
1: 0 -> 1 mv=S: 0 1 1 1 1 1 0 0

result: 0 1 1 1 1 1 1 0


In [173]:
UN_TIMES_TWO = {
    0: {
        0: (0, 0, "R"), 
        1: (1, 0, "R"),
    },
    1: {
        0: (2, 1, "L"), 
        1: (1, 1, "R"),
    },
    2: {
        0: (3, 0, "R"), 
        1: (4, 0, "R"),
    },
    3: {
        0: (0, 1, "S"), 
        1: (3, 1, "R"),
    },
    4: {
        0: (5, 1, "L"), 
        1: (4, 1, "R"),
    },
    5: {
        0: (2, 1, "L"), 
        1: (5, 1, "L"),
    },
}
un_times_two = TuringMachine(UN_TIMES_TWO)

In [174]:
tape = Tape([0, 1])
run_turing_machine(un_times_two, tape)

0: 0 -> 0 mv=R: 0 1
0: 1 -> 0 mv=R: 0 1
1: 0 -> 1 mv=L: 0 0 0
2: 0 -> 0 mv=R: 0 0 1
3: 1 -> 1 mv=R: 0 0 1
3: 0 -> 1 mv=S: 0 0 1 0

result: 0 0 1 1 0


In [175]:
tape = Tape([0, 1, 1, 1, 0])
tape = Tape([1, 1, 1])
run_turing_machine(un_times_two, tape)

0: 1 -> 0 mv=R: 1 1 1
1: 1 -> 1 mv=R: 0 1 1
1: 1 -> 1 mv=R: 0 1 1
1: 0 -> 1 mv=L: 0 1 1 0
2: 1 -> 0 mv=R: 0 1 1 1
4: 1 -> 1 mv=R: 0 1 0 1
4: 0 -> 1 mv=L: 0 1 0 1 0
5: 1 -> 1 mv=L: 0 1 0 1 1
5: 0 -> 1 mv=L: 0 1 0 1 1
2: 1 -> 0 mv=R: 0 1 1 1 1
4: 1 -> 1 mv=R: 0 0 1 1 1
4: 1 -> 1 mv=R: 0 0 1 1 1
4: 1 -> 1 mv=R: 0 0 1 1 1
4: 0 -> 1 mv=L: 0 0 1 1 1 0
5: 1 -> 1 mv=L: 0 0 1 1 1 1
5: 1 -> 1 mv=L: 0 0 1 1 1 1
5: 1 -> 1 mv=L: 0 0 1 1 1 1
5: 0 -> 1 mv=L: 0 0 1 1 1 1
2: 0 -> 0 mv=R: 0 1 1 1 1 1
3: 1 -> 1 mv=R: 0 1 1 1 1 1
3: 1 -> 1 mv=R: 0 1 1 1 1 1
3: 1 -> 1 mv=R: 0 1 1 1 1 1
3: 1 -> 1 mv=R: 0 1 1 1 1 1
3: 1 -> 1 mv=R: 0 1 1 1 1 1
3: 0 -> 1 mv=S: 0 1 1 1 1 1 0

result: 0 1 1 1 1 1 1 0


In [176]:
def int2bin(x):
    return [int(i) for i in bin(x)[2:]]


def bin2int(X):
    return sum(2**i if x == 1 else 0 for i, x in enumerate(reversed(X)))


def int2xnbin(X):
    y = []
    for x in X:
        for n in binary(x):
            if n == 0:
                y.append(0)
            elif n == 1:
                y.append(1)
                y.append(0)
        y.append(1)
        y.append(1)
        y.append(0)
    return y


def xnbin2int(y):
    i, x, X = 0, [], []
    while i < len(y):
        if y[i] == 0:
            x.append(0)
            i += 1
        elif y[i] == 1:
            i += 1
            if y[i] == 0:
                x.append(1)
                i += 1
            else:
                X.append(bin2int(x))
                i += 2
                x = []
    return X


In [177]:
x = 167
assert x == bin2int(int2bin(x))

int2exbin([x])
assert [x] == xnbin2int(int2xnbin([x]))

X = [6, 8]
int2exbin(X)
assert X == xnbin2int(int2xnbin(X))


[1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0]

[1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0]

In [178]:
XN_TIMES_TWO = {
    0: {
        0: (0, 0, "R"), 
        1: (1, 0, "R"),
    },
    1: {
        0: (0, 1, "R"), 
        1: (2, 0, "R"),
    },
    2: {
        0: (3, 1, "R"), 
    },
    3: {
        0: (0, 1, "S"),
    },
}
xn_times_two = TuringMachine(XN_TIMES_TWO)

In [179]:
x = 167
tape = Tape(int2exbin([x]))
run_turing_machine(xn_times_two, tape, max_iterations=50)
xnbin2int(tape)

0: 1 -> 0 mv=R: 1 0 0 1 0 0 0 1 0 1 0 1 0 1 1 0
1: 0 -> 1 mv=R: 0 0 0 1 0 0 0 1 0 1 0 1 0 1 1 0
0: 0 -> 0 mv=R: 0 1 0 1 0 0 0 1 0 1 0 1 0 1 1 0
0: 1 -> 0 mv=R: 0 1 0 1 0 0 0 1 0 1 0 1 0 1 1 0
1: 0 -> 1 mv=R: 0 1 0 0 0 0 0 1 0 1 0 1 0 1 1 0
0: 0 -> 0 mv=R: 0 1 0 0 1 0 0 1 0 1 0 1 0 1 1 0
0: 0 -> 0 mv=R: 0 1 0 0 1 0 0 1 0 1 0 1 0 1 1 0
0: 1 -> 0 mv=R: 0 1 0 0 1 0 0 1 0 1 0 1 0 1 1 0
1: 0 -> 1 mv=R: 0 1 0 0 1 0 0 0 0 1 0 1 0 1 1 0
0: 1 -> 0 mv=R: 0 1 0 0 1 0 0 0 1 1 0 1 0 1 1 0
1: 0 -> 1 mv=R: 0 1 0 0 1 0 0 0 1 0 0 1 0 1 1 0
0: 1 -> 0 mv=R: 0 1 0 0 1 0 0 0 1 0 1 1 0 1 1 0
1: 0 -> 1 mv=R: 0 1 0 0 1 0 0 0 1 0 1 0 0 1 1 0
0: 1 -> 0 mv=R: 0 1 0 0 1 0 0 0 1 0 1 0 1 1 1 0
1: 1 -> 0 mv=R: 0 1 0 0 1 0 0 0 1 0 1 0 1 0 1 0
2: 0 -> 1 mv=R: 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0
3: 0 -> 1 mv=S: 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 1 0

result: 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 1 1 0


[334]